Neon was retaining WAL and denying edit perms because its excess WAL was taking up too much storage, so I had to download the pgdump. 
This script reuploads it.

First step: Parse backup dump into JSON and upload in batches

In [7]:
import os
import json
import re

# Get the correct path to backup.sql in the root folder
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one level up
SQL_FILE = os.path.join(BASE_DIR, "backup.sql")
JSONL_FILE = os.path.join(BASE_DIR, "backup.jsonl")

print(f"Looking for backup.sql at: {SQL_FILE}")  # Debugging line

# Check if the file exists before proceeding
if not os.path.exists(SQL_FILE):
    raise FileNotFoundError(f"❌ Error: {SQL_FILE} not found. Are you running the script from the correct directory?")

print("✅ backup.sql found, proceeding with conversion...")

def stream_sql_to_json(input_sql, output_jsonl, batch_size=5000):
    """
    Converts SQL INSERT statements to JSONL in batches.
    """
    batch_count = 0
    batch = []  # ✅ Ensure batch is always initialized

    with open(input_sql, "r") as f, open(output_jsonl, "w") as f_out:
        sql_data = f.read()

        # Extract INSERT statements
        insert_statements = re.findall(r"INSERT INTO (\w+) VALUES (.*?);", sql_data, re.DOTALL)

        for table, values in insert_statements:
            values = values.split("),(")
            values = [v.strip("()") for v in values]

            for row in values:
                row_values = row.split(",")  # Assuming simple CSV structure
                batch.append(row_values)

                # Write in batches
                if len(batch) >= batch_size:
                    json.dump({table: batch}, f_out)
                    f_out.write("\n")
                    batch = []  # ✅ Reset batch correctly
                    batch_count += 1

        # ✅ Final write for any remaining data
        if batch:
            json.dump({table: batch}, f_out)
            batch_count += 1

    print(f"✅ JSON batch conversion complete: {batch_count} batches written.")

# Run conversion
stream_sql_to_json(SQL_FILE, JSONL_FILE)


Looking for backup.sql at: /Users/rohansharma/Desktop/Code/phil_rag/backup.sql
✅ backup.sql found, proceeding with conversion...
✅ JSON batch conversion complete: 0 batches written.
